# Assigment 3

In [20]:
import igl
import numpy as np
import scipy as sp
import meshplot as mp

from typing import Optional, Any

In [69]:
v, f = igl.read_triangle_mesh("data/cow.off")
mp.plot(v, f, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

# Vertex normal

In [124]:
# Standard face normal
def compute_standard_normals(vertices: list[Any], faces: list[Any]) -> list[Any]:
    """ Computes the unweighted average of surrounding face normals """
    return igl.per_vertex_normals(vertices, faces)

mp.plot(v, f, n=compute_standard_normals(v,f), shading={"flat": False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [139]:
# Area-weighted face normal
def compute_area_weighted_normals(vertices: list[Any], faces: list[Any]) -> list[Any]:
    """ Computes the area weighted average of surrounding face normals """
    return igl.per_vertex_normals(vertices, faces, weighting=igl.PER_VERTEX_NORMALS_WEIGHTING_TYPE_AREA )

mp.plot(v, f, n=compute_area_weighted_normals(v,f), shading={"flat": False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [143]:
# Mean-curvature normal
def compute_mean_curvature_normals(vertices: list[Any], faces: list[Any], eps: int) -> list[Any]:
    """ Uses thecotangent-weighted Laplacian to the mesh vertex positions to compute the normal weighted proportionally to mean curvature """
    
    # Step 1: Construct a Cotangent stiffness matrix (discrete laplacian)
    cotangent_matrix = igl.cotmatrix(vertices, faces)
   
    # Step 2: Construct the area matrix using VORONOI
    mass_matrix = igl.massmatrix(vertices, faces)
    
    minv = sp.sparse.diags(1 / mass_matrix.diagonal())
    hn = -minv.dot(cotangent_matrix.dot(vertices))
    h = np.linalg.norm(hn, axis=1)
    
    sn = compute_standard_normals(vertices, faces)
    wn = compute_area_weighted_normals(vertices, faces)

    normals:list[Any] = list()
    a = b = 0
    for idx, val in enumerate(h):
        if (np.linalg.norm(val) > eps):
            normals.append(sn[idx])
            a += 1
        else:
            normals.append(wn[idx])
            b += 1
    
    print(f"a: {a}, b:{b}")

    return np.array(normals) 

    
n = compute_mean_curvature_normals(v, f, 10)
mp.plot(v, f, n=n, shading={"flat": False})


a: 2178, b:584


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [131]:
# PCA normal
def compute_pca_normals(vertices: list[Any], faces: list[Any]) -> list[Any]:
    """ """
    
    normals = np.array([])
    return normals

n = compute_pca_normals(v, f)
mp.plot(v, f, n=n, shading={"flat": False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [ ]:
# Quadratic fitting normal


# Curvature

In [ ]:
# Gaussian curvature

In [ ]:
# Principal curvature

# Smoothing with the Laplacian

In [ ]:
from scipy.sparse.linalg import spsolve
import scipy.sparse as sp

In [ ]:
# Explicit laplacian

In [ ]:
# Implicit laplacian
